<a href="https://colab.research.google.com/github/RajaOsama08/Stock-Market-Prediction/blob/main/Stock_market_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
#import lib for K-NN
from sklearn.neighbors import KNeighborsRegressor
#import lib for XGBoost
from xgboost import XGBRegressor

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
#import lib for LSTM model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
# import lib for random forest model
from sklearn.ensemble import RandomForestRegressor



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Define the file paths
file_paths = {
    'AAPL': '/content/drive/MyDrive/stock_data_set/AAPL.csv',
    'CSCO': '/content/drive/MyDrive/stock_data_set/CSCO.csv',
    'INTC': '/content/drive/MyDrive/stock_data_set/INTC.csv',
    'NVDA': '/content/drive/MyDrive/stock_data_set/NVDA.csv',
    'SAP': '/content/drive/MyDrive/stock_data_set/SAP.csv'
}

# Load the datasets into pandas DataFrames
dataframes = {symbol: pd.read_csv(path) for symbol, path in file_paths.items()}

# Display the first few rows of each DataFrame
for symbol, df in dataframes.items():
    print(f"First few rows of {symbol} DataFrame:")
    print(df.head())
    print("\n")

# Display the first few rows of the DataFrame
print(df.head())

First few rows of AAPL DataFrame:
         Date       Open       High        Low      Close  Adj Close  \
0  2014-06-02  22.641430  22.672501  22.232143  22.451786  19.859396   
1  2014-06-03  22.445000  22.812143  22.437500  22.769285  20.140236   
2  2014-06-04  22.765715  23.138929  22.718214  23.029285  20.370214   
3  2014-06-05  23.078571  23.191786  22.950357  23.119642  20.450136   
4  2014-06-06  23.210714  23.259287  23.016787  23.056070  20.393915   

      Volume  
0  369350800  
1  292709200  
2  335482000  
3  303805600  
4  349938400  


First few rows of CSCO DataFrame:
         Date       Open       High        Low      Close  Adj Close    Volume
0  2014-06-02  24.639999  24.799999  24.559999  24.780001  18.135729  20393300
1  2014-06-03  24.750000  24.820000  24.450001  24.680000  18.062546  23339100
2  2014-06-04  24.610001  24.650000  24.400000  24.500000  17.930799  20635100
3  2014-06-05  24.620001  24.740000  24.410000  24.700001  18.077187  23320300
4  2014-06-0

In [5]:
df1 = dataframes['AAPL']
df2 = dataframes['CSCO']
df3 = dataframes['INTC']
df4 = dataframes['NVDA']
df5 = dataframes['SAP']
print(df1)

            Date        Open        High         Low       Close   Adj Close  \
0     2014-06-02   22.641430   22.672501   22.232143   22.451786   19.859396   
1     2014-06-03   22.445000   22.812143   22.437500   22.769285   20.140236   
2     2014-06-04   22.765715   23.138929   22.718214   23.029285   20.370214   
3     2014-06-05   23.078571   23.191786   22.950357   23.119642   20.450136   
4     2014-06-06   23.210714   23.259287   23.016787   23.056070   20.393915   
...          ...         ...         ...         ...         ...         ...   
2513  2024-05-24  188.820007  190.580002  188.039993  189.979996  189.979996   
2514  2024-05-28  191.509995  193.000000  189.100006  189.990005  189.990005   
2515  2024-05-29  189.610001  192.250000  189.509995  190.289993  190.289993   
2516  2024-05-30  190.759995  192.179993  190.630005  191.289993  191.289993   
2517  2024-05-31  191.440002  192.570007  189.910004  192.250000  192.250000   

         Volume  
0     369350800  
1  

In [6]:
df1['Date'] = pd.to_datetime(df1['Date'])
print('first print',df1)
print('\n')
df1.set_index('Date', inplace=True)
print('second print',df1)
print('\n')

first print            Date        Open        High         Low       Close   Adj Close  \
0    2014-06-02   22.641430   22.672501   22.232143   22.451786   19.859396   
1    2014-06-03   22.445000   22.812143   22.437500   22.769285   20.140236   
2    2014-06-04   22.765715   23.138929   22.718214   23.029285   20.370214   
3    2014-06-05   23.078571   23.191786   22.950357   23.119642   20.450136   
4    2014-06-06   23.210714   23.259287   23.016787   23.056070   20.393915   
...         ...         ...         ...         ...         ...         ...   
2513 2024-05-24  188.820007  190.580002  188.039993  189.979996  189.979996   
2514 2024-05-28  191.509995  193.000000  189.100006  189.990005  189.990005   
2515 2024-05-29  189.610001  192.250000  189.509995  190.289993  190.289993   
2516 2024-05-30  190.759995  192.179993  190.630005  191.289993  191.289993   
2517 2024-05-31  191.440002  192.570007  189.910004  192.250000  192.250000   

         Volume  
0     369350800  
1  

**Handling missing values**

In [7]:
# Check for missing values
missing_values = df1.isnull().sum()
print(missing_values)

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


**Feature Engineerig**



In [8]:
# Moving averages
df1['MA10'] = df1['Adj Close'].rolling(window=10).mean()
df1['MA50'] = df1['Adj Close'].rolling(window=50).mean()

# Daily returns
print('Daily returns')
df1['Daily Return'] = df1['Adj Close'].pct_change()

# Volatility
print('volatility')
df1['Volatility'] = df1['Daily Return'].rolling(window=10).std()

print(df1)

Daily returns
volatility
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2014-06-02   22.641430   22.672501   22.232143   22.451786   19.859396   
2014-06-03   22.445000   22.812143   22.437500   22.769285   20.140236   
2014-06-04   22.765715   23.138929   22.718214   23.029285   20.370214   
2014-06-05   23.078571   23.191786   22.950357   23.119642   20.450136   
2014-06-06   23.210714   23.259287   23.016787   23.056070   20.393915   
...                ...         ...         ...         ...         ...   
2024-05-24  188.820007  190.580002  188.039993  189.979996  189.979996   
2024-05-28  191.509995  193.000000  189.100006  189.990005  189.990005   
2024-05-29  189.610001  192.250000  189.509995  190.289993  190.289993   
2024-05-30  190.759995  192.179993  190.630005  191.289993  191.289993   
2024-05-31  191.440002  192.570007  189.910004  192.250000  192.250000   

            

**Dropping NaN**

In [9]:
print(pd.isna(df1).sum().sum())
# Drop rows with NaNs
df1.dropna(inplace=True)

69


In [10]:
scaler = MinMaxScaler()

# Scaling the features except for the target variable (Adj Close)
df1[['Open', 'High', 'Low', 'Close', 'Volume', 'MA10', 'MA50', 'Daily Return', 'Volatility']] = scaler.fit_transform(
    df1[['Open', 'High', 'Low', 'Close', 'Volume', 'MA10', 'MA50', 'Daily Return', 'Volatility']])

**Setting data for model prepration**

In [11]:
# Define features and target variable
features = ['Open', 'High', 'Low', 'Close', 'Volume', 'MA10', 'MA50', 'Daily Return', 'Volatility']
X = df1[features]
y = df1['Adj Close']

**Data Sampling for test-train**

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [13]:
# K-NN model
knn_model = KNeighborsRegressor()

# Hyperparameter tuning using GridSearchCV
HyperParams = {'n_neighbors': range(1, 20)}
grid_search_cv = GridSearchCV(knn_model, HyperParams, cv=5, scoring='neg_mean_squared_error')
grid_search_cv.fit(X_train, y_train)

# K-NN model best_estimator
best_knn_est = grid_search_cv.best_estimator_

# Model Train on training data set
best_knn_est.fit(X_train, y_train)

# Performing prediction on test data set
y_pred = best_knn_est.predict(X_test)

# Model Evaluation
K_mse = mean_squared_error(y_test, y_pred)
rmse = K_mse ** 0.5

print(f"Best K-NN model: {best_knn_est}")
print(f"RMSE on test set: {rmse}")

# Predicting the Adj Close price for the next day
next_day_features_KNN = X.iloc[-1].values.reshape(1, -1)
next_day_prediction_KNN = best_knn_est.predict(next_day_features_KNN)
print(f"Predicted Adj Close price for the next day: {next_day_prediction_KNN[0]}")

Best K-NN model: KNeighborsRegressor(n_neighbors=1)
RMSE on test set: 7.096278183634583
Predicted Adj Close price for the next day: 177.203201


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


In [14]:
# Step 4: XGBoost Model
xgb_model = XGBRegressor(objective='reg:squarederror')
xgb_model.fit(X_train, y_train)

# Predict on the test set
y_pred = xgb_model.predict(X_test)

# Evaluate the model
XGB_mse = mean_squared_error(y_test, y_pred)
rmse = XGB_mse ** 0.5

print(f"RMSE on test set: {rmse}")

# Predicting the Adj Close price for the next day
next_day_features_XGB = pd.DataFrame([X.iloc[-1]], columns=features)
next_day_features_scaled = scaler.transform(next_day_features_XGB)
next_day_prediction_XGB = xgb_model.predict(next_day_features_scaled)
print(f"Predicted Adj Close price for the next day: {next_day_prediction_XGB[0]}")

RMSE on test set: 6.467728803667712
Predicted Adj Close price for the next day: 21.71510887145996


**Implementing LSTM models**

In [18]:
# Reshape input data for LSTM in [samples, time steps, features]
X_train_LSTM = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
X_test_LSTM = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))

# LSTM model build
model_LSTM = Sequential()
model_LSTM.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_LSTM.shape[1], X_test_LSTM.shape[2])))
model_LSTM.add(Dropout(0.2))
model_LSTM.add(LSTM(units=50, return_sequences=False))
model_LSTM.add(Dropout(0.2))
model_LSTM.add(Dense(units=1))

# model compliation
model_LSTM.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model_LSTM.fit(X_train_LSTM, y_train, epochs=50, batch_size=32)

Epoch 1/50
62/62 [==============================] - 4s 5ms/step - loss: 5975.1367
Epoch 2/50
62/62 [==============================] - 0s 5ms/step - loss: 5256.6660
Epoch 3/50
62/62 [==============================] - 0s 5ms/step - loss: 4485.9419
Epoch 4/50
62/62 [==============================] - 0s 5ms/step - loss: 4080.3743
Epoch 5/50
62/62 [==============================] - 0s 5ms/step - loss: 3810.6255
Epoch 6/50
62/62 [==============================] - 0s 5ms/step - loss: 3582.2463
Epoch 7/50
62/62 [==============================] - 0s 5ms/step - loss: 3408.8562
Epoch 8/50
62/62 [==============================] - 0s 5ms/step - loss: 3251.4253
Epoch 9/50
62/62 [==============================] - 0s 5ms/step - loss: 3101.5879
Epoch 10/50
62/62 [==============================] - 0s 5ms/step - loss: 2980.5042
Epoch 11/50
62/62 [==============================] - 0s 5ms/step - loss: 2860.0315
Epoch 12/50
62/62 [==============================] - 0s 5ms/step - loss: 2760.5964
Epoch 13/50
6

**Implementing Random forest model**

In [21]:
# LSTM model prediction
train_LSTM_prediction = model_LSTM.predict(X_train_LSTM)
test_LSTM_prediction = model_LSTM.predict(X_test_LSTM)

# Feature prediction LSTM

RF_train_X = X_train.copy()
RF_test_X = X_test.copy()
RF_train_X['LSTM_Pred'] = train_LSTM_prediction
RF_test_X['LSTM_Pred'] = test_LSTM_prediction

# Train Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(RF_train_X, y_train)

16/16 [==============================] - 0s 3ms/step


RandomForestRegressor(random_state=42)

**Hybrid prediction LSTM-RF**

In [22]:
# Random Forest model prediction
train_rf_prediction = rf_model.predict(RF_train_X)
test_rf_prediction = rf_model.predict(RF_test_X)

# LSTM-RF Combine predictions
train_predictions = (train_LSTM_prediction.flatten() + train_rf_prediction) / 2
test_predictions = (test_LSTM_prediction.flatten() + test_rf_prediction) / 2

train_mse = mean_squared_error(y_train, train_predictions)
test_mse = mean_squared_error(y_test, test_predictions)
